# DeepAR - Experiments

### DeepAR (Salinas et al., 2017)

The **DeepAR** algorithm uses autoregressive features that are used as the inputs to a recurrent neural network. The RNN uses LSTM cells as default but can also changed to GRU cells. In both cases previous time points are taken as input. 

- [`DeepAREestimator`](https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html)

In [2]:
!pip install gluonts

    100% |████████████████████████████████| 296kB 7.5MB/s ta 0:00:011
    100% |████████████████████████████████| 10.4MB 4.5MB/s eta 0:00:01
    100% |████████████████████████████████| 28.4MB 1.5MB/s eta 0:00:01
    100% |████████████████████████████████| 81kB 33.1MB/s ta 0:00:01
    100% |████████████████████████████████| 4.8MB 7.4MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 31.3MB/s ta 0:00:01
    100% |████████████████████████████████| 194kB 15.0MB/s ta 0:00:01
  Running setup.py bdist_wheel for holidays ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/40/a9/2b/94ac5464363d37564a87dc93a9d21a5850aac14a4608197003
  Running setup.py bdist_wheel for ujson ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
Successfully built holidays ujson
  Found existing installation: pandas 0.24.2
    Uninstalling pandas-0.24.2:
      Successfully uninstalled pandas-0.24.2
You are using pip v

In [3]:
# standard imports
import numpy as np
import pandas as pd

# json
import json

# gluon data
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# model imports
# from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint


### define main function ### 
def main(data="m4_weekly", seed=42, epochs=100, batches=50):
    
    dataset = get_dataset(data, regenerate=False)
    mx.random.seed(seed)
    np.random.seed(seed)
    
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,      
        num_batches_per_epoch=batches,      
        learning_rate=1e-3,
    )
    
    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True   
    )
        
    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    output["epochs"] = epochs
    return(output)


INFO:root:Using CPU


In [3]:
results = []

for i in range(100, 401, 50):
    print("Epochs:", i)
    res = main(data="m4_daily", seed=42, epochs=i, batches=100)
    pprint(res)
    results.append(res)
    
results

Epochs: 100


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:07<00:00, 13.02it/s, avg_epoch_loss=7.26]
INFO:root:Epoch[0] Elapsed time 7.686 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.258919
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 12.05it/s, avg_epoch_loss=6.44]
INFO:root:Epoch[1] Elapsed time 8.305 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.435382
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 11.80it/s, avg_epoch_loss=6.45]
INFO:root:Epoch[2] Elapsed time 8.482 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.454049
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 12.41it/s, avg_epoch_loss=6.31]
INFO:root:Epoch[3] Elapsed time 8.060 seco

{'MASE': 4.6888278,
 'MSIS': 58.54613315,
 'epochs': 100,
 'sMAPE': 0.04235474,
 'wQuantileLoss[0.5]': 0.03813074,
 'wQuantileLoss[0.9]': 0.01942635}
Epochs: 150


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:07<00:00, 13.47it/s, avg_epoch_loss=7.26]
INFO:root:Epoch[0] Elapsed time 7.432 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.258919
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 12.31it/s, avg_epoch_loss=6.44]
INFO:root:Epoch[1] Elapsed time 8.127 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.435382
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 12.49it/s, avg_epoch_loss=6.45]
INFO:root:Epoch[2] Elapsed time 8.015 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.454049
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 12.09it/s, avg_epoch_loss=6.31]
INFO:root:Epoch[3] Elapsed time 8.281 seco

{'MASE': 3.34701839,
 'MSIS': 40.38508533,
 'epochs': 150,
 'sMAPE': 0.03185894,
 'wQuantileLoss[0.5]': 0.02774475,
 'wQuantileLoss[0.9]': 0.01301838}
Epochs: 200


...

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:07<00:00, 12.66it/s, avg_epoch_loss=7.26]
INFO:root:Epoch[0] Elapsed time 7.906 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.258919
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 12.13it/s, avg_epoch_loss=6.44]
INFO:root:Epoch[1] Elapsed time 8.251 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.435382
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 11.99it/s, avg_epoch_loss=6.45]
INFO:root:Epoch[2] Elapsed time 8.349 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.454049
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 12.36it/s, avg_epoch_loss=6.31]
INFO:root:Epoch[3] Elapsed time 8.093 seco

{'MASE': 3.38032501,
 'MSIS': 40.94456104,
 'epochs': 350,
 'sMAPE': 0.03213667,
 'wQuantileLoss[0.5]': 0.02768916,
 'wQuantileLoss[0.9]': 0.01282163}
Epochs: 400


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 12.43it/s, avg_epoch_loss=7.26]
INFO:root:Epoch[0] Elapsed time 8.054 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.258919
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 11.70it/s, avg_epoch_loss=6.44]
INFO:root:Epoch[1] Elapsed time 8.551 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.435382
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:09<00:00, 11.01it/s, avg_epoch_loss=6.45]
INFO:root:Epoch[2] Elapsed time 9.093 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.454049
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 11.23it/s, avg_epoch_loss=6.31]
INFO:root:Epoch[3] Elapsed time 8.909 seco

{'MASE': 3.37004807,
 'MSIS': 40.71273682,
 'epochs': 400,
 'sMAPE': 0.03205514,
 'wQuantileLoss[0.5]': 0.02758192,
 'wQuantileLoss[0.9]': 0.01278391}


[{'MASE': 4.6888278,
  'sMAPE': 0.04235474,
  'MSIS': 58.54613315,
  'wQuantileLoss[0.5]': 0.03813074,
  'wQuantileLoss[0.9]': 0.01942635,
  'epochs': 100},
 {'MASE': 3.34701839,
  'sMAPE': 0.03185894,
  'MSIS': 40.38508533,
  'wQuantileLoss[0.5]': 0.02774475,
  'wQuantileLoss[0.9]': 0.01301838,
  'epochs': 150},
 {'MASE': 3.34129322,
  'sMAPE': 0.03175431,
  'MSIS': 40.38673493,
  'wQuantileLoss[0.5]': 0.02770024,
  'wQuantileLoss[0.9]': 0.01305936,
  'epochs': 200},
 {'MASE': 3.607182,
  'sMAPE': 0.03346469,
  'MSIS': 42.24145034,
  'wQuantileLoss[0.5]': 0.03064602,
  'wQuantileLoss[0.9]': 0.01383277,
  'epochs': 250},
 {'MASE': 3.37422146,
  'sMAPE': 0.03209932,
  'MSIS': 40.7909502,
  'wQuantileLoss[0.5]': 0.0276275,
  'wQuantileLoss[0.9]': 0.0128443,
  'epochs': 300},
 {'MASE': 3.38032501,
  'sMAPE': 0.03213667,
  'MSIS': 40.94456104,
  'wQuantileLoss[0.5]': 0.02768916,
  'wQuantileLoss[0.9]': 0.01282163,
  'epochs': 350},
 {'MASE': 3.37004807,
  'sMAPE': 0.03205514,
  'MSIS': 40.

In [2]:
results = []

for i in range(100, 401, 50):
    print("Epochs:", i)
    res = main(data="m4_daily", seed=43, epochs=i, batches=100)
    pprint(res)
    results.append(res)
    
results

Epochs: 100


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:09<00:00, 10.41it/s, avg_epoch_loss=6.94]
INFO:root:Epoch[0] Elapsed time 9.612 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.943348
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:09<00:00, 11.07it/s, avg_epoch_loss=6.54]
INFO:root:Epoch[1] Elapsed time 9.042 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.544661
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:12<00:00,  8.24it/s, avg_epoch_loss=6.43]
INFO:root:Epoch[2] Elapsed time 12.149 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.434662
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:12<00:00,  7.84it/s, avg_epoch_loss=6.28]
INFO:root:Epoch[3] Elapsed time 12.754 se

{'MASE': 3.76198041,
 'MSIS': 42.13462873,
 'epochs': 100,
 'sMAPE': 0.03513189,
 'wQuantileLoss[0.5]': 0.03183971,
 'wQuantileLoss[0.9]': 0.01504344}
Epochs: 150


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:12<00:00,  8.21it/s, avg_epoch_loss=6.94]
INFO:root:Epoch[0] Elapsed time 12.183 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.943348
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:16<00:00,  6.10it/s, avg_epoch_loss=6.54]
INFO:root:Epoch[1] Elapsed time 16.396 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.544661
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:09<00:00, 10.08it/s, avg_epoch_loss=6.43]
INFO:root:Epoch[2] Elapsed time 9.924 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.434662
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:10<00:00,  9.94it/s, avg_epoch_loss=6.28]
INFO:root:Epoch[3] Elapsed time 10.069 s

{'MASE': 3.809087,
 'MSIS': 46.15768663,
 'epochs': 150,
 'sMAPE': 0.03519001,
 'wQuantileLoss[0.5]': 0.03329824,
 'wQuantileLoss[0.9]': 0.01593606}
Epochs: 200


...

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:10<00:00,  9.18it/s, avg_epoch_loss=6.94]
INFO:root:Epoch[0] Elapsed time 10.904 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.943348
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:09<00:00, 10.27it/s, avg_epoch_loss=6.54]
INFO:root:Epoch[1] Elapsed time 9.743 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.544661
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:09<00:00, 10.57it/s, avg_epoch_loss=6.43]
INFO:root:Epoch[2] Elapsed time 9.463 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.434662
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:10<00:00,  9.87it/s, avg_epoch_loss=6.28]
INFO:root:Epoch[3] Elapsed time 10.139 se

{'MASE': 3.53595053,
 'MSIS': 42.99588989,
 'epochs': 350,
 'sMAPE': 0.0332005,
 'wQuantileLoss[0.5]': 0.03082195,
 'wQuantileLoss[0.9]': 0.01440264}
Epochs: 400


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:12<00:00,  7.70it/s, avg_epoch_loss=6.94]
INFO:root:Epoch[0] Elapsed time 13.002 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.943348
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:11<00:00,  8.75it/s, avg_epoch_loss=6.54]
INFO:root:Epoch[1] Elapsed time 11.444 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.544661
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:13<00:00,  7.67it/s, avg_epoch_loss=6.43]
INFO:root:Epoch[2] Elapsed time 13.050 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.434662
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:13<00:00,  7.31it/s, avg_epoch_loss=6.28]
INFO:root:Epoch[3] Elapsed time 13.689 

{'MASE': 3.53992429,
 'MSIS': 43.04474713,
 'epochs': 400,
 'sMAPE': 0.03322258,
 'wQuantileLoss[0.5]': 0.03089263,
 'wQuantileLoss[0.9]': 0.0144801}


[{'MASE': 3.76198041,
  'sMAPE': 0.03513189,
  'MSIS': 42.13462873,
  'wQuantileLoss[0.5]': 0.03183971,
  'wQuantileLoss[0.9]': 0.01504344,
  'epochs': 100},
 {'MASE': 3.809087,
  'sMAPE': 0.03519001,
  'MSIS': 46.15768663,
  'wQuantileLoss[0.5]': 0.03329824,
  'wQuantileLoss[0.9]': 0.01593606,
  'epochs': 150},
 {'MASE': 3.54183074,
  'sMAPE': 0.03324842,
  'MSIS': 43.06691573,
  'wQuantileLoss[0.5]': 0.03086203,
  'wQuantileLoss[0.9]': 0.01445214,
  'epochs': 200},
 {'MASE': 3.53705613,
  'sMAPE': 0.03320447,
  'MSIS': 42.80685746,
  'wQuantileLoss[0.5]': 0.03080833,
  'wQuantileLoss[0.9]': 0.01434419,
  'epochs': 250},
 {'MASE': 3.53236466,
  'sMAPE': 0.03318516,
  'MSIS': 42.98269824,
  'wQuantileLoss[0.5]': 0.03079269,
  'wQuantileLoss[0.9]': 0.01442702,
  'epochs': 300},
 {'MASE': 3.53595053,
  'sMAPE': 0.0332005,
  'MSIS': 42.99588989,
  'wQuantileLoss[0.5]': 0.03082195,
  'wQuantileLoss[0.9]': 0.01440264,
  'epochs': 350},
 {'MASE': 3.53992429,
  'sMAPE': 0.03322258,
  'MSIS': 

In [2]:
res = main(data="m4_daily", seed=44, epochs=100, batches=100)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 11.93it/s, avg_epoch_loss=7.12]
INFO:root:Epoch[0] Elapsed time 8.391 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.123951
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 11.89it/s, avg_epoch_loss=6.44]
INFO:root:Epoch[1] Elapsed time 8.422 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.443972
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 11.51it/s, avg_epoch_loss=6.42]
INFO:root:Epoch[2] Elapsed time 8.696 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.415508
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 11.84it/s, avg_epoch_loss=6.35]
INFO:root:Epoch[3] Elapsed time 8.450 seco

{'MASE': 3.74909462,
 'MSIS': 44.04607147,
 'epochs': 100,
 'sMAPE': 0.03473306,
 'wQuantileLoss[0.5]': 0.03155759,
 'wQuantileLoss[0.9]': 0.01412131}


In [6]:
res = main(data="m4_daily", seed=44, epochs=150, batches=100)
pprint(res)

In [ ]:
res = main(data="m4_daily", seed=44, epochs=200, batches=100)
pprint(res)

In [ ]:
res = main(data="m4_daily", seed=44, epochs=250, batches=100)
pprint(res)

In [ ]:
res = main(data="m4_daily", seed=44, epochs=300, batches=100)
pprint(res)

In [ ]:
res = main(data="m4_daily", seed=44, epochs=350, batches=100)
pprint(res)

In [ ]:
res = main(data="m4_daily", seed=44, epochs=400, batches=100)
pprint(res)

In [4]:
res = main(data="m4_daily", seed=42, epochs=100, batches=200)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 200/200 [00:27<00:00,  7.32it/s, avg_epoch_loss=6.85]
INFO:root:Epoch[0] Elapsed time 27.336 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.847151
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:25<00:00,  7.96it/s, avg_epoch_loss=6.38]
INFO:root:Epoch[1] Elapsed time 25.147 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.382643
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:25<00:00,  7.94it/s, avg_epoch_loss=6.24]
INFO:root:Epoch[2] Elapsed time 25.191 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.242576
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:21<00:00,  9.49it/s, avg_epoch_loss=6.13]
INFO:root:Epoch[3] Elapsed time 21.091 

{'MASE': 5.4063357,
 'MSIS': 77.86403918,
 'epochs': 100,
 'sMAPE': 0.04754383,
 'wQuantileLoss[0.5]': 0.04598875,
 'wQuantileLoss[0.9]': 0.02633989}


In [2]:
res = main(data="m4_daily", seed=42, epochs=150, batches=200)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 200/200 [00:21<00:00,  9.20it/s, avg_epoch_loss=6.85]
INFO:root:Epoch[0] Elapsed time 21.751 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.847151
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:23<00:00,  8.62it/s, avg_epoch_loss=6.38]
INFO:root:Epoch[1] Elapsed time 23.202 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.382643
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:33<00:00,  5.94it/s, avg_epoch_loss=6.24]
INFO:root:Epoch[2] Elapsed time 33.702 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.242576
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:41<00:00,  4.82it/s, avg_epoch_loss=6.13]
INFO:root:Epoch[3] Elapsed time 41.465 

{'MASE': 3.31045227,
 'MSIS': 40.79797344,
 'epochs': 150,
 'sMAPE': 0.0316434,
 'wQuantileLoss[0.5]': 0.02787393,
 'wQuantileLoss[0.9]': 0.01310215}


In [3]:
res = main(data="m4_daily", seed=42, epochs=200, batches=200)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 200/200 [00:19<00:00, 10.37it/s, avg_epoch_loss=6.85]
INFO:root:Epoch[0] Elapsed time 19.292 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.847151
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:27<00:00,  7.17it/s, avg_epoch_loss=6.38]
INFO:root:Epoch[1] Elapsed time 27.896 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.382643
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:17<00:00, 11.35it/s, avg_epoch_loss=6.24]
INFO:root:Epoch[2] Elapsed time 17.634 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.242576
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:17<00:00, 11.48it/s, avg_epoch_loss=6.13]
INFO:root:Epoch[3] Elapsed time 17.428 

{'MASE': 3.52237985,
 'MSIS': 41.86274029,
 'epochs': 200,
 'sMAPE': 0.03313088,
 'wQuantileLoss[0.5]': 0.02980716,
 'wQuantileLoss[0.9]': 0.01329117}


In [4]:
res = main(data="m4_daily", seed=42, epochs=250, batches=200)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 200/200 [00:24<00:00,  8.23it/s, avg_epoch_loss=6.85]
INFO:root:Epoch[0] Elapsed time 24.307 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.847151
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:22<00:00,  8.95it/s, avg_epoch_loss=6.38]
INFO:root:Epoch[1] Elapsed time 22.346 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.382643
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:23<00:00,  8.66it/s, avg_epoch_loss=6.24]
INFO:root:Epoch[2] Elapsed time 23.102 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.242576
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:21<00:00,  9.31it/s, avg_epoch_loss=6.13]
INFO:root:Epoch[3] Elapsed time 21.498 

{'MASE': 3.63534742,
 'MSIS': 42.68165088,
 'epochs': 250,
 'sMAPE': 0.03389154,
 'wQuantileLoss[0.5]': 0.03102737,
 'wQuantileLoss[0.9]': 0.01364864}


In [5]:
res = main(data="m4_daily", seed=42, epochs=300, batches=200)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
 50%|████▉     | 99/200 [00:14<00:14,  6.97it/s, avg_epoch_loss=7.14]


KeyboardInterrupt: 

In [ ]:
res = main(data="m4_daily", seed=42, epochs=350, batches=200)
pprint(res)

In [ ]:
res = main(data="m4_daily", seed=42, epochs=400, batches=200)
pprint(res)

In [4]:
res = main(data="m4_daily", seed=43, epochs=100, batches=200)
pprint(res)

INFO:root:downloading and processing m4_daily


saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily/train/data.json
saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily/test/data.json


INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 200/200 [00:13<00:00, 14.42it/s, avg_epoch_loss=6.74]
INFO:root:Epoch[0] Elapsed time 13.879 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.744002
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:14<00:00, 14.24it/s, avg_epoch_loss=6.37]
INFO:root:Epoch[1] Elapsed time 14.055 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.367438
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:14<00:00, 14.21it/s, avg_epoch_loss=6.21]
INFO:root:Epoch[2] Elapsed time 14.080 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.212882
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:13<00:00, 14.38it/s, avg_epoch_loss=6.12]
INFO:root:Epoch[3] Elapsed time 13.914 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=6.118385
INFO:root:Epoch[4] Learning ra

{'MASE': 4.42535449,
 'MSIS': 52.09988401,
 'epochs': 100,
 'sMAPE': 0.03932475,
 'wQuantileLoss[0.5]': 0.0372285,
 'wQuantileLoss[0.9]': 0.01659874}


In [5]:
res = main(data="m4_daily", seed=43, epochs=150, batches=200)
pprint(res)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 200/200 [00:13<00:00, 14.65it/s, avg_epoch_loss=6.74]
INFO:root:Epoch[0] Elapsed time 13.655 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.744002
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:14<00:00, 14.18it/s, avg_epoch_loss=6.37]
INFO:root:Epoch[1] Elapsed time 14.109 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.367438
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:14<00:00, 14.13it/s, avg_epoch_loss=6.21]
INFO:root:Epoch[2] Elapsed time 14.156 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.212882
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:14<00:00, 14.24it/s, avg_epoch_loss=6.12]
INFO:root:Epoch[3] Elapsed time 14.0

{'MASE': 3.3112046,
 'MSIS': 38.7626711,
 'epochs': 150,
 'sMAPE': 0.03175051,
 'wQuantileLoss[0.5]': 0.02784221,
 'wQuantileLoss[0.9]': 0.01241791}


In [6]:
res = main(data="m4_daily", seed=43, epochs=200, batches=200)
pprint(res)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 200/200 [00:13<00:00, 14.56it/s, avg_epoch_loss=6.74]
INFO:root:Epoch[0] Elapsed time 13.744 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.744002
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:14<00:00, 14.08it/s, avg_epoch_loss=6.37]
INFO:root:Epoch[1] Elapsed time 14.212 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.367438
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:14<00:00, 14.01it/s, avg_epoch_loss=6.21]
INFO:root:Epoch[2] Elapsed time 14.281 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.212882
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:14<00:00, 14.14it/s, avg_epoch_loss=6.12]
INFO:root:Epoch[3] Elapsed time 14.1

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-01f20515d05f>", line 1, in <module>
    res = main(data="m4_daily", seed=43, epochs=200, batches=200)
  File "<ipython-input-3-ba25b09ea1a9>", line 49, in main
    predictor = estimator.train(dataset.train)
  File "/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/gluonts/model/estimator.py", line 203, in train
    return self.train_model(training_data).predictor
  File "/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/gluonts/model/estimator.py", line 189, in train_model
    train_iter=training_data_loader,
  File "/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/gluonts/trainer/_base.py", line 266, in __call__
    epoch_loss.update(None, preds=loss)


KeyboardInterrupt: 

In [ ]:
res = main(data="m4_daily", seed=43, epochs=250, batches=200)
pprint(res)

In [ ]:
res = main(data="m4_daily", seed=43, epochs=300, batches=200)
pprint(res)

In [ ]:
res = main(data="m4_daily", seed=43, epochs=350, batches=200)
pprint(res)

In [ ]:
res = main(data="m4_daily", seed=43, epochs=400, batches=200)
pprint(res)

In [5]:
# results = []

# for i in range(100, 401, 50):
#     print("Epochs:", i)
#     res = main(data="m4_daily", seed=44, epochs=i, batches=100)
#     pprint(res)
#     results.append(res)
    
# results

In [ ]:
# results = []

# for i in range(100, 401, 50):
#     print("Epochs:", i)
#     res = main(data="m4_daily", seed=42, epochs=i, batches=200)
#     pprint(res)
#     results.append(res)
    
# results

In [ ]:
# results = []

# for i in range(100, 401, 50):
#     print("Epochs:", i)
#     res = main(data="m4_daily", seed=43, epochs=i, batches=200)
#     pprint(res)
#     results.append(res)
    
# results

In [ ]:
# results = []

# for i in range(100, 401, 50):
#     print("Epochs:", i)
#     res = main(data="m4_daily", seed=44, epochs=i, batches=200)
#     pprint(res)
#     results.append(res)
    
# results